<a href="https://colab.research.google.com/github/HansHenseler/masdav2022/blob/main/Part_7_Exercises_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing and visualising geolocation data

Part 7

Master of Advanced Studies in Digital Forensics & Cyber Investigation

Data Analytics and Visualization for Digital Forensics

(c) Hans Henseler, 2022

# Part 7

Importing libraries

In [ ]:
# Need to install various packages. 
#
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install geopandas
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 2s (928 kB/s)
Selecting previously unselected p

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import plotly_express as px

In [ ]:
!sudo apt-get install -y exiftool

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libimage-exiftool-perl' instead of 'exiftool'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libarchive-zip-perl libmime-charset-perl libposix-strptime-perl libsombok3
  libunicode-linebreak-perl
Suggested packages:
  libencode-hanextra-perl libpod2-base-perl
The following NEW packages will be installed:
  libarchive-zip-perl libimage-exiftool-perl libmime-charset-perl
  libposix-strptime-perl libsombok3 libunicode-linebreak-perl
0 upgraded, 6 newly installed, 0 to remove and 20 not upgraded.
Need to get 3,395 kB of archives.
After this operation, 20.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libarchive-zip-perl all 1.60-1ubuntu0.1 [84.6 kB]
Get:2 http://archive.ub

# Exercises

## 1 Analyse pictures from an Android 10 phone for geotags and plot on a map

In [ ]:
# Download Android_10.zip from Digital Corpora (takes app 7 minutes to download in Google Colab)
#
!wget "https://digitalcorpora.s3.amazonaws.com/corpora/mobile/android_10/Android_10.zip"

--2022-08-25 13:42:28--  https://digitalcorpora.s3.amazonaws.com/corpora/mobile/android_10/Android_10.zip
Resolving digitalcorpora.s3.amazonaws.com (digitalcorpora.s3.amazonaws.com)... 52.218.234.91
Connecting to digitalcorpora.s3.amazonaws.com (digitalcorpora.s3.amazonaws.com)|52.218.234.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10476724716 (9.8G) [application/zip]
Saving to: ‘Android_10.zip’

Android_10.zip      100%[===================>]   9.76G  21.2MB/s    in 8m 45s  

2022-08-25 13:51:14 (19.0 MB/s) - ‘Android_10.zip’ saved [10476724716/10476724716]



In [ ]:
# In this zip there is a zip file containing a logical extraction of the file system
# /Android 10 Image with Documentation/Non-Cellebrite Extraction/Pixel 3.zip
# extract this zip file (takes about 4 minutes, use -j to ignore file path)

!unzip -j Android_10.zip  Android\ 10\ Image\ with\ Documentation/Non-Cellebrite\ Extraction/Pixel\ 3.zip

Archive:  Android_10.zip
  inflating: Pixel 3.zip             


In [ ]:
# investigate the contents of this zip file for jpg files, extract, run the exiftool and plot on a map
# hint: use the unzip -l option (e.g. to find *.jpg)

!unzip -l "Pixel 3.zip" | grep jpg

    22062  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/InApp_Countdown_Timer (2).jpg
    15012  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/nonsale_avatar.jpg
   135034  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/nonsale_defaultinterstitial_extra101.jpg
    41336  2020-01-31 21:15   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-3692132869510698294.jpg
    13696  2020-01-31 21:14   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-5812264224591469663.jpg
    29512  2020-01-31 21:13   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-6982875141504110754.jpg
   577563  2020-01-29 11:55   Pixel 3/data/data/com.google.vr.apps.ornament/app_preview_temp/shared/com.google.vr.apps.ornament.content.stickers.spark/spark_preview.jpg


In [ ]:
# create a folder images in your colab (manually). Export all *.jpg and ignor paths (-j option)
# to this images folder (-d ./images). Note: choose N for None if unzip asks if it must overwrite a file.
#
!unzip -j -d ./images "Pixel 3.zip" "*.jpg"

Archive:  Pixel 3.zip
  inflating: ./images/InApp_Countdown_Timer (2).jpg  
  inflating: ./images/nonsale_avatar.jpg  
  inflating: ./images/nonsale_defaultinterstitial_extra101.jpg  
  inflating: ./images/thisisdfir@gmail.com-3692132869510698294.jpg  
  inflating: ./images/thisisdfir@gmail.com-5812264224591469663.jpg  
  inflating: ./images/thisisdfir@gmail.com-6982875141504110754.jpg  
  inflating: ./images/spark_preview.jpg  
 extracting: ./images/6BIqfFk.jpg    
 extracting: ./images/CXW0PEe.jpg    
  inflating: ./images/82150035d650b73619f729f8797a2979.jpg  
  inflating: ./images/043a8097f43c6fb006dd4a97ee9447be.jpg  
  inflating: ./images/04a860c62dd3c32bc987962bf50826f0.jpg  
  inflating: ./images/0cf03778a47919ffa242abbb61045546.jpg  
  inflating: ./images/15093b05575b105cfb87ca1f977ba155.jpg  
  inflating: ./images/235cf218066aa4e50fab6637df280536.jpg  
  inflating: ./images/2f8011301fcab6a4e63fd3d3f58bdf2e.jpg  
  inflating: ./images/40a02bf1a684b89c672336492d738f56.jpg  
  i

In [ ]:
!exiftool -csv -c '%+.6f' -gpslatitude -gpslongitude -GPSLatitudeRef -GPSLongitudeRef -r -ext jpg -T ./images > data.csv

Error: File format error - ./images/._thumbnail.jpg


In [ ]:
# we can ignore the file format error
#
!cat data.csv

SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
./images/CXW0PEe.jpg,-,-,-,-
./images/share-bg.321973c9.jpg,-,-,-,-
./images/100.jpg,-,-,-,-
./images/ec7d2b808b41220218b77a00bf74981e.jpg,-,-,-,-
./images/task-header-bg.c3fe62a2.jpg,-,-,-,-
./images/82150035d650b73619f729f8797a2979.jpg,-,-,-,-
./images/image-2020-02-08-15-35-37.jpg,-,-,-,-
./images/IMG-20200208-WA0000.jpg,-,-,-,-
./images/23.jpg,-,-,-,-
./images/f0e91b6172596ba0ffbb526a0eb7abf1.jpg,-,-,-,-
./images/nonsale_defaultinterstitial_extra101.jpg,-,-,-,-
./images/f878f5245587260295c9177235487a77.jpg,-,-,-,-
./images/06ceeadd-995d-4e3c-be2b-2a0b26dec374.jpg,-,-,-,-
./images/f65f74c98e70a327b25baada680aa2bd.jpg,-,-,-,-
./images/3-cf1a125e4c75c154fe28f1d9e2ca33d8.jpg,-,-,-,-
./images/c782e5730b5a6867937583273ee0a475.jpg,-,-,-,-
./images/1581642305950.jpg,-,-,-,-
./images/82.jpg,-,-,-,-
./images/f152cd39d46e6c1c2668422e32227c0e.jpg,-,-,-,-
./images/signal-2020-02-01-151617.jpg,-,-,-,-
./images/8a0665d96cecb6779e5

In [ ]:
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SourceFile       131 non-null    object
 1   GPSLatitude      131 non-null    object
 2   GPSLongitude     131 non-null    object
 3   GPSLatitudeRef   131 non-null    object
 4   GPSLongitudeRef  131 non-null    object
dtypes: object(5)
memory usage: 5.2+ KB


In [ ]:
df

,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
0,./images/64.jpg,-,-,-,-
1,./images/40ejXKPJbEpvicHxBCaTeuOMQBZ.jpg,-,-,-,-
2,./images/share-bg.321973c9.jpg,-,-,-,-
3,./images/7d6a48ef8e6f9662983bbe8d8989d721.jpg,-,-,-,-
4,./images/CXW0PEe.jpg,-,-,-,-
...,...,...,...,...,...
126,./images/291_reduced.jpg,-,-,-,-
127,./images/83436cf6965cc3a28777664c474c3cf4.jpg,-,-,-,-
128,./images/9-9be4865a8c4bca2241463164cf5d1f3e.jpg,-,-,-,-
129,./images/7-6e2bf1bbb768daf2c68f218edefadd66.jpg,-,-,-,-


In [ ]:
df = df[df.GPSLatitude !=  '-'].reset_index()
df

,index,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
0,34,./images/IMG_20200209_132353.jpg,+35.660981,-78.879897,North,West
1,46,./images/IMG_20190917_082637.jpg,+35.735647,-78.636367,North,West
2,112,./images/IMG_20200213_195030.jpg,+35.658867,-78.874217,North,West


In [ ]:
jpg_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.GPSLatitude, df.GPSLongitude))
jpg_gdf

,index,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef,geometry
0,34,./images/IMG_20200209_132353.jpg,+35.660981,-78.879897,North,West,POINT (35.66098 -78.87990)
1,46,./images/IMG_20190917_082637.jpg,+35.735647,-78.636367,North,West,POINT (35.73565 -78.63637)
2,112,./images/IMG_20200213_195030.jpg,+35.658867,-78.874217,North,West,POINT (35.65887 -78.87422)


In [ ]:
map = folium.Map(location = [35.660981,-78.879897,], tiles='OpenStreetMap' , width='75%',height='75%',zoom_start = 8)

# Create a geometry list from the GeoDataFrame
jpg_list = [[point.xy[0][0], point.xy[1][0]] for point in jpg_gdf.geometry ]

# Iterate through list and add a marker for each host in jpg_gdf

i = 0
for coordinates in jpg_list:
    map.add_child(folium.Marker(location = coordinates, popup=jpg_gdf.SourceFile[i]))
    i = i+1

map

## 2 Can you find any other sources besides jpg files that contain geo location and can you also extract  and visualise that information?

hint: consider sqlite files from Android apps

In [ ]:
# One sqlite database is the myplaces db from Google maps
#
# gmm_myplaces.db
#

!unzip -l "Pixel 3.zip" | grep gmm_myplaces

    40960  2020-02-09 14:45   Pixel 3/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db
    40960  2020-02-09 14:45   Pixel 3/sbin/.magisk/mirror/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db


In [ ]:
!unzip -j "Pixel 3.zip"  "Pixel 3/sbin/.magisk/mirror/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db"

Archive:  Pixel 3.zip
  inflating: gmm_myplaces.db         


In [ ]:
import sqlite3

gmmdb = sqlite3.connect('gmm_myplaces.db')

In [ ]:
df = pd.read_sql_query("SELECT * from sync_item", gmmdb)
df

,corpus,key_string,timestamp,merge_key,feature_fprint,latitude,longitude,is_local,sync_item
0,8,1:0,1554407343604,0,None,35734602,-78636654,0,b'\x12\x031:02\xd1\x01\n\x04\x08\x01\x10\x00\x...


In [ ]:
# need to divide by 1000000 because there is no decimal point

gmmdb_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.latitude/1000000, df.longitude/1000000))
gmmdb_gdf

,corpus,key_string,timestamp,merge_key,feature_fprint,latitude,longitude,is_local,sync_item,geometry
0,8,1:0,1554407343604,0,None,35734602,-78636654,0,b'\x12\x031:02\xd1\x01\n\x04\x08\x01\x10\x00\x...,POINT (35.73460 -78.63665)


In [ ]:
# the google timestamp is unix epoch in milliseconds. If we /1000 we can use datetime.fromtimestamp to get a datetime

from datetime import datetime
print(datetime.fromtimestamp(gmmdb_gdf.timestamp[0]/1000).strftime('Google %Y-%m-%d %H:%M:%S'))

Google 2019-04-04 19:49:03


In [ ]:
map = folium.Map(location = [gmmdb_gdf.geometry[0].xy[0][0],gmmdb_gdf.geometry[0].xy[1][0]], tiles='OpenStreetMap' , width='75%',height='75%',zoom_start = 8)

# Create a geometry list from the GeoDataFrame
gmmdb_list = [[point.xy[0][0], point.xy[1][0]] for point in gmmdb_gdf.geometry ]

# Iterate through list and add a marker for each host in jpg_gdf
i = 0
for coordinates in gmmdb_list:
    popup = datetime.fromtimestamp(gmmdb_gdf.timestamp[i]/1000).strftime('Google myplaces timestamp %Y-%m-%d %H:%M:%S')
    map.add_child(folium.Marker(location = coordinates, popup=popup))
    i = i+1
map